In [36]:
import pandas as pd
from collections import Counter
import re
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

In [37]:
queries_df = pd.read_csv("/content/Queries.csv")
print(queries_df.head())

                                 Top queries  Clicks  Impressions     CTR  \
0                number guessing game python    5223        14578  35.83%   
1                        thecleverprogrammer    2809         3456  81.28%   
2           python projects with source code    2077        73380   2.83%   
3  classification report in machine learning    2012         4959  40.57%   
4                      the clever programmer    1931         2528  76.38%   

   Position  
0      1.61  
1      1.02  
2      5.94  
3      1.28  
4      1.09  


Exploratory Data Analysis

In [38]:
print(queries_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Top queries  1000 non-null   object 
 1   Clicks       1000 non-null   int64  
 2   Impressions  1000 non-null   int64  
 3   CTR          1000 non-null   object 
 4   Position     1000 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 39.2+ KB
None


Convert the CTR column from a percentage string to a float:

In [39]:
def clean_and_split(query):
    words = re.findall(r'\b[a-zA-Z]+\b', query.lower())
    return words

In [40]:
# Split each query into words and count the frequency of each word
word_counts = Counter()
for query in queries_df['Top queries']:
    word_counts.update(clean_and_split(query))

In [41]:
word_freq_df = pd.DataFrame(word_counts.most_common(20), columns=['Word', 'Frequency'])

# Plotting the word frequencies
fig = px.bar(word_freq_df, x='Word', y='Frequency', title='Top 20 Most Common Words in Search Queries')
fig.show()

Now, let’s have a look at the top queries by clicks and impressions:


In [42]:
# Top queries by Clicks and Impressions
top_queries_clicks_vis = queries_df.nlargest(10, 'Clicks')[['Top queries', 'Clicks']]
top_queries_impressions_vis = queries_df.nlargest(10, 'Impressions')[['Top queries', 'Impressions']]

In [43]:
# Plotting
fig_clicks = px.bar(top_queries_clicks_vis, x='Top queries', y='Clicks', title='Top Queries by Clicks')
fig_impressions = px.bar(top_queries_impressions_vis, x='Top queries', y='Impressions', title='Top Queries by Impressions')
fig_clicks.show()
fig_impressions.show()

Now, let’s analyze the queries with the highest and lowest CTRs:

In [44]:
print(queries_df.columns)


Index(['Top queries', 'Clicks', 'Impressions', 'CTR', 'Position'], dtype='object')


In [45]:
print(queries_df['CTR'].head(10))


0    35.83%
1    81.28%
2     2.83%
3    40.57%
4    76.38%
5    21.38%
6     25.9%
7    10.71%
8    28.73%
9     37.7%
Name: CTR, dtype: object


In [46]:
# Show original text (not converted)
print(queries_df['CTR'].astype(str).head(20))


0     35.83%
1     81.28%
2      2.83%
3     40.57%
4     76.38%
5     21.38%
6      25.9%
7     10.71%
8     28.73%
9      37.7%
10    28.17%
11     5.76%
12    16.97%
13    42.45%
14    19.63%
15      3.1%
16    21.81%
17    25.19%
18    26.99%
19    17.43%
Name: CTR, dtype: object


In [47]:
import re

def clean_ctr(value):
    if pd.isna(value):
        return None
    value = str(value)
    value = re.sub(r'[^\d,.\-]', '', value)  # keep only digits, dot, comma, minus
    value = value.replace(',', '.')          # convert comma to dot
    try:
        return float(value)
    except:
        return None

queries_df['CTR'] = queries_df['CTR'].apply(clean_ctr)


In [48]:
print(queries_df['CTR'].head(15))
print(queries_df['CTR'].dtype)


0     35.83
1     81.28
2      2.83
3     40.57
4     76.38
5     21.38
6     25.90
7     10.71
8     28.73
9     37.70
10    28.17
11     5.76
12    16.97
13    42.45
14    19.63
Name: CTR, dtype: float64
float64


In [49]:
top_ctr_vis = queries_df.nlargest(10, 'CTR')[['Top queries', 'CTR']]
bottom_ctr_vis = queries_df.nsmallest(10, 'CTR')[['Top queries', 'CTR']]

import plotly.express as px
fig_top_ctr = px.bar(top_ctr_vis, x='Top queries', y='CTR', title='Top Queries by CTR')
fig_bottom_ctr = px.bar(bottom_ctr_vis, x='Top queries', y='CTR', title='Bottom Queries by CTR')
fig_top_ctr.show()
fig_bottom_ctr.show()


Now, let’s have a look at the correlation between different metrics:

In [50]:
# Correlation matrix visualization
correlation_matrix = queries_df[['Clicks', 'Impressions', 'CTR', 'Position']].corr()
fig_corr = px.imshow(correlation_matrix, text_auto=True, title='Correlation Matrix')
fig_corr.show()

**Detecting Anomalies in Search** **Queries**

In [51]:
from sklearn.ensemble import IsolationForest

In [52]:
# Selecting relevant features
features = queries_df[['Clicks', 'Impressions', 'CTR', 'Position']]

In [53]:
# Initializing Isolation Forest
iso_forest = IsolationForest(n_estimators=100, contamination=0.01)  # contamination is the expected proportion of outliers

In [54]:
# Fitting the model
iso_forest.fit(features)

# Predicting anomalies
queries_df['anomaly'] = iso_forest.predict(features)

# Filtering out the anomalies
anomalies = queries_df[queries_df['anomaly'] == -1]

In [55]:
print(anomalies[['Top queries', 'Clicks', 'Impressions', 'CTR', 'Position']])

                                  Top queries  Clicks  Impressions    CTR  \
0                 number guessing game python    5223        14578  35.83   
1                         thecleverprogrammer    2809         3456  81.28   
2            python projects with source code    2077        73380   2.83   
3   classification report in machine learning    2012         4959  40.57   
4                       the clever programmer    1931         2528  76.38   
11                          clever programmer    1243        21566   5.76   
15                 rock paper scissors python    1111        35824   3.10   
21                      classification report     933        39896   2.34   
34                   machine learning roadmap     708        42715   1.66   
82                                   r2 score     367        56322   0.65   

    Position  
0       1.61  
1       1.02  
2       5.94  
3       1.28  
4       1.09  
11      4.82  
15      7.19  
21      7.53  
34      8.97  
82

The anomalies in our search query data are not just outliers. They are indicators of potential areas for growth, optimization, and strategic focus. These anomalies are reflecting emerging trends or areas of growing interest. Staying responsive to these trends will help in maintaining and growing the website’s relevance and user engagement.

**Summary**

So, Search Queries Anomaly Detection means identifying queries that are outliers according to their performance metrics. It is valuable for businesses to spot potential issues or opportunities, such as unexpectedly high or low CTRs. I hope you liked this article on Search Queries Anomaly Detection with Machine Learning using Python.